In [122]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.ndimage import rotate

In [123]:
(X_train, y_train) , (X_test, y_test) = keras.datasets.mnist.load_data()

In [124]:
X_train = X_train / 255
X_test = X_test / 255

In [125]:
# Define rotation angles
rotation_angles = [15, 30, 45, 60, 90]

# Function to rotate images
def generate_rotated_images(images, angles):
    rotated_images = {}
    for angle in angles:
        rotated_images[angle] = np.array([rotate(img, angle, reshape=False) for img in images])
    return rotated_images

# Generate rotated versions of the training and test sets
rotated_X_train = generate_rotated_images(X_train, rotation_angles)
rotated_X_test = generate_rotated_images(X_test, rotation_angles)

# Now, to integrate these rotated images with the existing data
# You could concatenate the rotated images and corresponding labels like so:

# Concatenate original and rotated images for training
X_train_expanded = np.concatenate([X_train] + [rotated_X_train[angle] for angle in rotation_angles])
y_train_expanded = np.concatenate([y_train] * (len(rotation_angles) + 1))

# Concatenate original and rotated images for testing
X_test_expanded = np.concatenate([X_test] + [rotated_X_test[angle] for angle in rotation_angles])
y_test_expanded = np.concatenate([y_test] * (len(rotation_angles) + 1))

# Now, X_train_expanded and X_test_expanded contain the original and rotated images
# y_train_expanded and y_test_expanded contain the corresponding labels

print("Expanded training set shape:", X_train_expanded.shape)
print("Expanded test set shape:", X_test_expanded.shape)

Expanded training set shape: (360000, 28, 28)
Expanded test set shape: (60000, 28, 28)


In [126]:
X_train_flattened = X_train_expanded.reshape(len(X_train_expanded), 28*28)
X_test_flattened = X_test_expanded.reshape(len(X_test_expanded), 28*28)

In [127]:
X_train_flattened.shape

(360000, 784)

In [141]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(784, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10,steps_per_epoch=3000)

Epoch 1/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9088 - loss: 0.3074
Epoch 2/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9769 - loss: 0.0752
Epoch 4/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9845 - loss: 0.0479
Epoch 6/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9891 - loss: 0.0334
Epoch 8/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9921 - loss: 0.0246
Epoch 10/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00


In [142]:
model.evaluate(X_test,y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9742 - loss: 0.0909


[0.07989715784788132, 0.9775999784469604]

In [143]:
y_predicted = model.predict(X_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [145]:
np.argmax(y_predicted[0])

7

In [146]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

In [147]:
OUTDIR = "./mnist_digits/model.weights.h5"
checkpoint_callback = ModelCheckpoint(
    OUTDIR, save_weights_only=True)

In [148]:
model.save('full_model.h5')